# Importing the data to a database

The dataset we could have found on the internet are very complete xls file contaiing all the load, wind and solar charge factor hourly data for several years for all the countries

In order to exploit it more efficiently we will normalize them to table indexed by Count, Year, Month, Day and Hour and import all of them to a database we can then query efficiently

In [1]:
import pandas as pd

## Consumption data

The loads dataset we found is almost indexed as we want it to but present the hour as columns. So we need to stack those into rows

In [16]:
conso = pd.read_excel('../data/Monthly-hourly-load-values_2006-2015.xlsx', skiprows=3)
conso.describe()

,Year,Month,Day,Coverage ratio,0,1,2,3,4,5,...,14,15,16,17,18,19,20,21,22,23
count,116802.000000,116802.000000,116802.000000,114913.000000,116793.000000,116791.000000,116481.000000,116788.000000,116790.000000,116790.000000,...,116792.000000,116791.000000,116792.000000,116792.000000,116792.000000,116792.000000,116792.000000,116792.000000,116792.000000,116781.000000
mean,2010.897005,6.519657,15.727693,99.478040,9099.260288,8713.666844,8438.263186,8287.352176,8310.414419,8611.124116,...,11139.868313,11023.640866,11032.864187,11236.337557,11428.538432,11412.349835,11170.579391,10797.352030,10450.106416,9798.138657
std,2.777437,3.447775,8.799310,1.956024,12975.508290,12568.190793,12165.012750,11911.894718,11969.661345,12471.540059,...,15942.673135,15717.151681,15666.198528,16031.221511,16422.617119,16293.352349,15753.571506,15156.484825,14886.410402,14021.211972
min,2006.000000,1.000000,1.000000,81.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2009.000000,4.000000,8.000000,100.000000,1169.000000,1094.000000,1054.000000,1036.000000,1047.000000,1127.000000,...,1512.000000,1495.000000,1494.000000,1513.000000,1565.000000,1576.000000,1573.000000,1508.000000,1395.000000,1281.000000
50%,2011.000000,7.000000,16.000000,100.000000,4345.000000,4156.000000,3998.000000,3899.000000,3881.000000,3997.000000,...,4992.000000,4981.000000,5027.000000,5127.000000,5233.500000,5304.000000,5276.000000,5167.000000,4960.500000,4650.000000
75%,2013.000000,10.000000,23.000000,100.000000,10000.000000,9483.500000,9199.000000,9070.000000,9080.000000,9315.000000,...,11427.000000,11375.500000,11373.500000,11557.250000,11655.250000,11570.750000,11250.500000,11001.000000,11018.250000,10664.000000
max,2015.000000,12.000000,31.000000,100.000000,88828.000000,91216.000000,89645.000000,87082.000000,86910.000000,90412.000000,...,94727.000000,92965.000000,92907.000000,95642.000000,102098.000000,99872.000000,95132.000000,90964.000000,93728.000000,92410.000000


In [17]:
conso.index.name='id'
pivot = conso.iloc[:,5:].stack()
pivot.name='Consumption (MWh)'
pivot=pd.DataFrame(pivot)
pivot['Hour'] = pivot.index.get_level_values(1)
conso = conso.iloc[:,:4].merge(pivot, on='id').set_index(['Country','Year', 'Month', 'Day', 'Hour'])

In [8]:
conso.head()

Consumption (MWh)
Country Year Month Day Hour                   
AT      2006 1     1   0                6297.0
                       1                6005.0
                       2                5743.0
                       3                5397.0
                       4                5213.0

## Wind and solar load factors
Wind and solar charge factor come in the same format with countries as columns so we just need to pivot those ones

In [9]:
wind = pd.read_excel('../data/EMHIRES_WIND_COUNTRY_June2019.xlsx')
wind.describe()


,Time_step,Year,Month,Day,Hour,AL,AT,BE,BG,BA,...,NL,NO,PL,PT,RO,RS,SE,SI,SK,UK
count,262968.000000,262968.000000,262968.000000,262968.000000,262968.0000,262968.000000,262968.000000,262968.000000,262968.000000,262968.000000,...,262968.000000,262968.000000,262968.000000,262968.000000,262968.000000,262968.000000,262968.000000,262968.000000,262968.000000,262968.000000
mean,131484.500000,2000.499681,6.523136,15.729032,11.5000,0.192005,0.284966,0.219247,0.175378,0.172726,...,0.257536,0.234455,0.241043,0.231411,0.206581,0.150402,0.260767,0.141935,0.167874,0.281156
std,75912.467132,8.655064,3.448653,8.799853,6.9222,0.189453,0.295231,0.225203,0.180194,0.179407,...,0.229632,0.138302,0.227140,0.200070,0.199103,0.160475,0.229128,0.222608,0.216641,0.210246
min,1.000000,1986.000000,1.000000,1.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000260,0.000000,0.000070,0.000000,0.000000,0.000440,0.000000,0.000000,0.000317
25%,65742.750000,1993.000000,4.000000,8.000000,5.7500,0.040500,0.042980,0.038390,0.040190,0.035000,...,0.051800,0.119130,0.062740,0.080390,0.058230,0.034500,0.079238,0.003990,0.002840,0.103094
50%,131484.500000,2000.000000,7.000000,16.000000,11.5000,0.131500,0.168685,0.134780,0.113120,0.108000,...,0.189280,0.214960,0.161840,0.164075,0.136680,0.094000,0.186475,0.041030,0.073130,0.229406
75%,197226.250000,2008.000000,10.000000,23.000000,17.2500,0.294500,0.452305,0.338400,0.250142,0.258500,...,0.433895,0.339190,0.358630,0.323300,0.291813,0.211500,0.385080,0.174660,0.251330,0.428451
max,262968.000000,2015.000000,12.000000,31.000000,23.0000,0.996008,0.999980,0.795510,0.896790,1.001500,...,0.712460,0.579430,0.911160,0.959080,0.992050,1.008000,0.999930,0.999640,0.883210,0.791400


In [10]:
wind.index.name = 'id'
pivot = wind.iloc[:,7:].stack()
pivot.name='Wind Charge Factor'
pivot=pd.DataFrame(pivot)
pivot['Country'] = pivot.index.get_level_values(1)
wind = wind.iloc[:,2:6].merge(pivot, on='id').set_index(['Country','Year', 'Month', 'Day', 'Hour'])

In [11]:
wind = wind.sort_index()
wind.head()

Wind Charge Factor
Country Year Month Day Hour                    
AT      1986 1     1   0                0.06084
                       1                0.05282
                       2                0.04276
                       3                0.02914
                       4                0.01780

In [13]:
solar = pd.read_excel('../data/EMHIRESPV_TSh_CF_Country_19862015.xlsx')
solar.index.name = 'id'
pivot = solar.iloc[:,7:].stack()
pivot.name='Solar Charge Factor'
pivot=pd.DataFrame(pivot)
pivot['Country'] = pivot.index.get_level_values(1)
solar = solar.iloc[:,2:6].merge(pivot, on='id').set_index(['Country','Year', 'Month', 'Day', 'Hour'])


In [14]:
all_data = conso.merge(wind, on =['Country','Year', 'Month', 'Day', 'Hour'])
all_data = all_data.merge(solar, on =['Country','Year', 'Month', 'Day', 'Hour'])
all_data.head()
#for  

Consumption (MWh)  Wind Charge Factor  \
Country Year Month Day Hour                                          
AT      2006 1     1   0                6297.0             0.48037   
                       1                6005.0             0.51660   
                       2                5743.0             0.43948   
                       3                5397.0             0.35967   
                       4                5213.0             0.36907   

                             Solar Charge Factor  
Country Year Month Day Hour                       
AT      2006 1     1   0                     0.0  
                       1                     0.0  
                       2                     0.0  
                       3                     0.0  
                       4                     0.0

In [15]:
for (country,year),data in all_data.groupby(level=[0,1]):
    print(country, year)
    data.reset_index(level=[0,1], drop=True).to_csv('../data/%s_%s.csv' % (country, year))

AT 2006
AT 2007
AT 2008
AT 2009
AT 2010
AT 2011
AT 2012
AT 2013
AT 2014
AT 2015
BA 2006
BA 2007
BA 2008
BA 2009
BA 2010
BA 2011
BA 2012
BA 2013
BA 2014
BA 2015
BE 2006
BE 2007
BE 2008
BE 2009
BE 2010
BE 2011
BE 2012
BE 2013
BE 2014
BE 2015
BG 2006
BG 2007
BG 2008
BG 2009
BG 2010
BG 2011
BG 2012
BG 2013
BG 2014
BG 2015
CH 2006
CH 2007
CH 2008
CH 2009
CH 2010
CH 2011
CH 2012
CH 2013
CH 2014
CH 2015
CY 2010
CY 2011
CY 2012
CY 2013
CY 2014
CY 2015
CZ 2006
CZ 2007
CZ 2008
CZ 2009
CZ 2010
CZ 2011
CZ 2012
CZ 2013
CZ 2014
CZ 2015
DE 2006
DE 2007
DE 2008
DE 2009
DE 2010
DE 2011
DE 2012
DE 2013
DE 2014
DE 2015
DK 2010
DK 2011
DK 2012
DK 2013
DK 2014
DK 2015
EE 2009
EE 2010
EE 2011
EE 2012
EE 2013
EE 2014
EE 2015
ES 2006
ES 2007
ES 2008
ES 2009
ES 2010
ES 2011
ES 2012
ES 2013
ES 2014
ES 2015
FI 2010
FI 2011
FI 2012
FI 2013
FI 2014
FI 2015
FR 2006
FR 2007
FR 2008
FR 2009
FR 2010
FR 2011
FR 2012
FR 2013
FR 2014
FR 2015
HR 2006
HR 2007
HR 2008
HR 2009
HR 2010
HR 2011
HR 2012
HR 2013
HR 2014
HR 2015
